In [1]:
import PIL.Image as Image
import cv2
import numpy as np

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers,losses
from tensorflow.keras.models import Sequential

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import zipfile

local_zip = '/content/drive/MyDrive/Colab Notebooks/archive.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import pathlib                               
train_dir = pathlib.Path("/content/dataset/training_set")


print(train_dir)

/content/dataset/training_set


In [5]:

list(train_dir.glob('*/*'))[:5] 

[PosixPath('/content/dataset/training_set/cats/cat.465.jpg'),
 PosixPath('/content/dataset/training_set/cats/cat.2164.jpg'),
 PosixPath('/content/dataset/training_set/cats/cat.2814.jpg'),
 PosixPath('/content/dataset/training_set/cats/cat.611.jpg'),
 PosixPath('/content/dataset/training_set/cats/cat.22.jpg')]

In [6]:
cat_train= list(train_dir.glob('cats/*'))
len(cat_train)

4000

In [7]:
image_count= len(list(train_dir.glob('*/*.jpg')))
image_count

8000

In [8]:
"""Image.open(str(cat_train[1500]))"""

'Image.open(str(cat_train[1500]))'

In [9]:
dog_train= list(train_dir.glob('dogs/*'))
len(cat_train)

4000

In [10]:
"""Image.open(str(dog_train[1569]))"""

'Image.open(str(dog_train[1569]))'

In [11]:
images_dict = {
    'cats': cat_train[:2200],
    'dogs': dog_train[:2200],
}

In [12]:
images_dict["dogs"][:5]

[PosixPath('/content/dataset/training_set/dogs/dog.3856.jpg'),
 PosixPath('/content/dataset/training_set/dogs/dog.310.jpg'),
 PosixPath('/content/dataset/training_set/dogs/dog.1005.jpg'),
 PosixPath('/content/dataset/training_set/dogs/dog.3461.jpg'),
 PosixPath('/content/dataset/training_set/dogs/dog.286.jpg')]

In [13]:
labels_dict = {
    'cats': 0,
    'dogs': 1,
    
}

In [14]:
img= cv2.imread(str(images_dict['cats'][0]))

In [15]:
img.shape

(374, 500, 3)

In [16]:

X, y = [], []
for name,images in images_dict.items():#returns key , path for each record
    for image in images:
        img=cv2.imread(str(image))
     
        resize_img=cv2.resize(img,(224,224))
       
        X.append(resize_img)
        y.append(labels_dict[name])

In [17]:
X = np.array(X)
y = np.array(y)

In [18]:
X.shape,y.shape

((4400, 224, 224, 3), (4400,))

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30 ,random_state=0)

In [20]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [21]:
model = Sequential([
                pretrained_model_without_top_layer,
                tf.keras.layers.Dense(2)
])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [23]:
# Scaling the image 
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [24]:
model.compile(
    optimizer="adam",
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc']
)
model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
97/97 [==============================] - 37s 43ms/step - loss: 0.1648 - acc: 0.9364
Epoch 2/5
97/97 [==============================] - 4s 40ms/step - loss: 0.0578 - acc: 0.9795
Epoch 3/5
97/97 [==============================] - 4s 41ms/step - loss: 0.0449 - acc: 0.9831
Epoch 4/5
97/97 [==============================] - 4s 40ms/step - loss: 0.0371 - acc: 0.9873
Epoch 5/5
97/97 [==============================] - 4s 40ms/step - loss: 0.0289 - acc: 0.9896


In [25]:
model.evaluate(X_test_scaled,y_test)

42/42 [==============================] - 2s 41ms/step - loss: 0.0382 - acc: 0.9894


[0.03824220597743988, 0.989393949508667]